In [35]:
using JuMP
using CPLEX
using Distributions
using LinearAlgebra
using Statistics
using Dates
using DataFrames
using SDDP
using Plots
import CSV
using JSON
try
    using Revise
catch e
    @warn "Error initializing Revise" exception=(e, catch_backtrace())
end

includet(pwd() * "\\Water_Regulation\\WaterRegulation.jl")
using .WaterRegulation

In [36]:
filepath_Ljungan = pwd() * "\\Water_Regulation\\TestDataWaterRegulation\\Ljungan.json"
filepath_prices = pwd() *  "\\Data\\Spot Prices\\prices_df.csv"
filepath_results = pwd() * "\\Results\\LambdaZero\\"
R, K, J = read_data(filepath_Ljungan)
print() 

j = J[1]
O = OtherParticipant(J, j , R)[1]
K_j = [j.plants[1]]
K_O = [O.plants[1]]
pj = j.participationrate
pO = O.participationrate


println("Participation rate $(j.name): \n ", pj)
println("Individual Reservoir of $(j.name): \n", j.individualreservoir)
println("Participation rate $(O.name): \n ", pO)
println("K_O : $(K_O) and spillage $(K_O[1].spillreference),\nK_j : $(K_j) and spillage $(K_j[1].spillreference)")

Stages = 7 # Days in Time Hoirzon <-> Stages
T = 24

# Balancing Penalties
mu_up = 1.0
mu_down = 0.3

Omega = [1.0]
P = [1 / length(Omega) for el in eachindex(Omega)]

Participation rate Sydkraft: 
 ________________________________
Flasjon  | 1.84    
Holsmjon | 0.0     

Individual Reservoir of Sydkraft: 
________________________________
Flasjon  | 15000.0 
Holsmjon | 18000.0 

Participation rate Other: 
 ________________________________
Flasjon  | 2.68    
Holsmjon | 2.68    

K_O : HydropowerPlant[Parteboda] and spillage 1.4,
K_j : HydropowerPlant[Flasjo] and spillage 0.58


1-element Vector{Float64}:
 1.0

### Short-Term Scheduling of Hydropower Plants

After Day-Ahead Market Bidding and Adjusted Flow Calculation, we have the opportunity to renominate.  
It does not matter wether the agent is anticipatory or nonanticipatory: In any case the players have to abide to the water regulation rules.  

The optimizers' goal is to disperse water correctly and solve the unit commitment problem of which power plants to use. Additionally, we minimize the amount of balancing we haave to do.  

Bidding is removed from the optimization model. We now only have to work with the known constraint of demand in the first stage:
$$
y_t = \sum\limits_{k \in \mathcal{K}} w_{k,t} + z^+_t - z^-_t 
$$

The subsequent stages don't have a known demand, as bidding is only done for the first stage. In the next stages we leave out the demand constraint.


The goal is to find an optimal nomination and production schedul. The nomination includes information of how we value the reservoirs' contents.   
In the objective we can either leave out or keep the revenues from day-ahead obligations. As they have to be fulfilled anyways it does not have any impact on the optimal solution.

##### Parameters
* Reference Flow by Water Regulation Company
* Day-Ahead Obligations for every hour
* Adjusted Flow -> Other's nomination

Power Swap and own nomination can also be extracted but are irrelevant as they will be redetermined.

In [37]:
Qref = Dict{Reservoir, Float64}(r => 0.8 for r in R)
QnomO = Dict{Reservoir, Float64}(r => 0.8 for r in R)
println(QnomO)
y::Vector{Float64} = 2 .* rand(24)


________________________________
Flasjon  | 0.8     
Holsmjon | 0.8     



24-element Vector{Float64}:
 0.08728672341471166
 0.2211949650547984
 0.2959583341231731
 1.9012547263861348
 1.027997717567679
 0.4591913815269548
 0.08784005771452708
 1.4864636376498201
 0.28702357698861647
 1.506130035815257
 ⋮
 0.4978934037427556
 0.48562335320691474
 1.4481881266272565
 1.0037364104861026
 0.12413934307411312
 0.22350728948279697
 0.3306717505053718
 1.0552550159323084
 0.13454778104872678

In [38]:
function subproblem_builder_short(subproblem::Model, node::Int64)
    # State Variables
    @variable(subproblem, 0 <= l[r = R] <= r.maxvolume, SDDP.State, initial_value = r.currentvolume)
    @variable(subproblem, lind[r = R], SDDP.State, initial_value = j.individualreservoir[r])
    @variable(subproblem, u_start[k = K], SDDP.State, initial_value = 0, Bin)
    # Control Variables
    @variable(subproblem, Qnom[r = R] >= 0)
    @variable(subproblem, Qadj[r = R] >= 0)
    @variable(subproblem, d[t = 1:T, k = K], Bin)
    @variable(subproblem, u[t = 1:T, k = K], Bin)
    @variable(subproblem, BALANCE_INDICATOR[r = R], Bin)
    @variable(subproblem, 0 <= w[t = 1:T, k = K] <= k.equivalent * k.spillreference)
    @variable(subproblem, z_up[t = 1:T] >= 0)
    @variable(subproblem, z_down[t = 1:T] >= 0)
    @variable(subproblem, 0 <= Qeff[t = 1:T, k = K] <= k.spillreference)
    @variable(subproblem, 0 <= Qreal[t = 1:T, r = R])
    @variable(subproblem, Pswap[r = R])
    @variable(subproblem, Pover[k = K_O] >= 0)
    @variable(subproblem, f[r = R] >= 0)
    @variable(subproblem, s[r = R] >= 0)
    # Transition Function
    @constraint(subproblem, balance[r = R], l[r].out == l[r].in - T * Qadj[r] + f[r] * T - s[r])
    @constraint(subproblem, balance_ind[r = R], lind[r].out == lind[r].in - T * (Qnom[r] - Qref[r])- s[r]) 
    @constraint(subproblem, startcond[k = K], u_start[k].in == u[1,k])
    @constraint(subproblem, endcond[k = K], u_start[k].out == u[T,k])
    # Constraints
    @constraint(subproblem, nbal1[r = R], BALANCE_INDICATOR[r] => {Qnom[r] <= Qref[r]})
    @constraint(subproblem, nbal2[r = R], !BALANCE_INDICATOR[r] => {0 <= lind[r].in})
    @constraint(subproblem, NoSpill[k = K], BALANCE_INDICATOR[k.reservoir] => {sum(Qnom[r_up] for r_up in find_us_reservoir(k.reservoir)) <= k.spillreference})
    @constraint(subproblem, adjustedflow[r = R], Qadj[r] == (Qnom[r] * pj[r] + QnomO[r] * pO[r]) / (pj[r] + pO[r]))
    @constraint(subproblem, powerswap[r = R], Pswap[r] == pj[r] * (Qnom[r] - Qadj[r]) - sum(Pover[k] for k in K_O))
    @constraint(subproblem, overnomination[k = K_O], Pover[k] >= k.equivalent * (Qadj[k.reservoir] - k.spillreference))
    @constraint(subproblem, nomination[r = R], sum(Qreal[t,r] for t in 1:T) == T * Qadj[r])
    @constraint(subproblem, obligation[t = 1:T], y[t]  == sum(w[t,k] for k in K) + sum(Pswap[r] for r in R) + z_up[t] - z_down[t])
    @constraint(subproblem, active[t = 1:T, k = K], w[t,k] <= u[t,k] * k.spillreference * k.equivalent)
    @constraint(subproblem, startup[t = 1:T-1, k = K], d[t,k] >= u[t+1,k] - u[t,k])
    @constraint(subproblem, production[t = 1:T, k = K], w[t,k] <= Qeff[t,k] * k.equivalent)
    @constraint(subproblem, realwater[t = 1:T, k = K], Qeff[t,k] <= sum(Qreal[t,r] for r in find_us_reservoir(k.reservoir)))
    SDDP.parameterize(subproblem, Omega, P) do om
        # We have to make sure that depending on the market clearing price, the coefficients are set accordingly.
        # The recourse action only applies to the real delivery, determined by the uncertain price. The other restricitions become inactive, else they make the problem infeasible.
        # The constraints that are relevant are maiintained in Scenario_Index for every current time step.
        for r in R
            JuMP.fix(f[r], om, force=true)
        end
        # Include only active variables in stageobjective
        @stageobjective(subproblem, sum(mu_up * z_up[t] - mu_down * z_down[t]  + S * sum(d[t,k] for k in K) for t in 1:T))
    end
    return
end

subproblem_builder_short (generic function with 1 method)

In [39]:
model_balancing = SDDP.LinearPolicyGraph(
    subproblem_builder_short,
    stages = Stages,
    sense = :Min,
    lower_bound = sum(y[t] * mu_down for t in 1:T),
    optimizer = CPLEX.Optimizer
)

A policy graph with 7 nodes.
 Node indices: 1, 2, 3, 4, 5, 6, 7


### Real Time Balancing and Water Scheduling

Even after going through two optimization problems, we are still not done yet.  
After all participants communicated their renomination, a new adjusted flow has been calculated.  
Although multiple renominations are theoretically possible, we choose to only have one round of nomination before and after market clearing each. That is because no new information outside of the water regulation procedure is gained.  

The last problem is simply a problem of real time balancing and unit commitment, as the amount of water to be discharged is determined. The Water Value function doesn't play a role anymore. We minimize costs:

$$
\min \; \rightarrow \; \sum\limits_{t \in \mathcal{T}} \left( \sum\limits_{k \in \mathcal{K}} ( S_k \cdot \delta_{k,t} ) +  \mu^+ z^+_t - \mu^- z^-_t  \right)
$$

As a Parameter we get:

* Adjusted Flow
* Power Swap

We Decide:

* Flow of water at every hour
* Balancing Decisions
* Startups of each power plant

In [40]:
Qadj = Dict{Reservoir, Float64}(r => 0.8 for r in R)
Pswap = Dict{Reservoir, Float64}(r => 0.0 for r in R)

Dict{Reservoir, Float64} with 2 entries:
  Flasjon  => 0.0
  Holsmjon => 0.0

In [41]:
function subproblem_builder_balancing(subproblem::Model, node::Int64)
    # State Variables
    @variable(subproblem, u_start[k = K], SDDP.State, initial_value = 0, Bin)
    # Control Variables
    @variable(subproblem, d[t = 1:T, k = K], Bin)
    @variable(subproblem, u[t = 1:T, k = K], Bin)
    @variable(subproblem, 0 <= w[t = 1:T, k = K] <= k.equivalent * k.spillreference)
    @variable(subproblem, z_up[t = 1:T] >= 0)
    @variable(subproblem, z_down[t = 1:T] >= 0)
    @variable(subproblem, 0 <= Qeff[t = 1:T, k = K] <= k.spillreference)
    @variable(subproblem, 0 <= Qreal[t = 1:T, r = R])
    # Transition Function
    @constraint(subproblem, startcond[k = K], u_start[k].in == u[1,k])
    @constraint(subproblem, endcond[k = K], u_start[k].out == u[T,k])
    # Constraints
    @constraint(subproblem, nomination[r = R], sum(Qreal[t,r] for t in 1:T) == T * Qadj[r])
    @constraint(subproblem, obligation[t = 1:T], y[t]  == sum(w[t,k] for k in K) + sum(Pswap[r] for r in R) + z_up[t] - z_down[t])
    @constraint(subproblem, active[t = 1:T, k = K], w[t,k] <= u[t,k] * k.spillreference * k.equivalent)
    @constraint(subproblem, startup[t = 1:T-1, k = K], d[t,k] >= u[t+1,k] - u[t,k])
    @constraint(subproblem, production[t = 1:T, k = K], w[t,k] <= Qeff[t,k] * k.equivalent)
    @constraint(subproblem, realwater[t = 1:T, k = K], Qeff[t,k] <= sum(Qreal[t,r] for r in find_us_reservoir(k.reservoir)))
    SDDP.parameterize(subproblem, Omega, P) do om
        # We have to make sure that depending on the market clearing price, the coefficients are set accordingly.
        # The recourse action only applies to the real delivery, determined by the uncertain price. The other restricitions become inactive, else they make the problem infeasible.
        # The constraints that are relevant are maiintained in Scenario_Index for every current time step.
        for r in R
            JuMP.fix(f[r], om.inflow, force=true)
            JuMP.set_normalized_rhs(adjustedflow[r], pO[r] * om.nomination)
        end
        # Include only active variables in stageobjective
        @stageobjective(subproblem, sum(mu_up * z_up[t] - mu_down * z_down[t]  + S * sum(d[t,k] for k in K) for t in 1:T))
    end
    return
end

subproblem_builder_balancing (generic function with 1 method)

In [42]:
model_balancing = SDDP.LinearPolicyGraph(
    subproblem_builder_balancing,
    stages = 1,
    sense = :Min,
    lower_bound = sum(y[t] * mu_down for t in 1:T),
    optimizer = CPLEX.Optimizer
)

A policy graph with 1 nodes.
 Node indices: 1
